In [1]:
import pickle
import numpy as np
import pandas as pd
import copy
import os
import sys
sys.path.append("/home/ubuntu/CE_scheme/")
from Schemas.imdb.schema import gen_imdb_schema
from Join_scheme.data_prepare import read_table_csv
from Join_scheme.bound import Bound_ensemble

In [2]:
import ast


OPS = {
    '>': np.greater,
    '<': np.less,
    '>=': np.greater_equal,
    '<=': np.less_equal,
    '=': np.equal,
    '==': np.equal
}


def process_condition(cond, tables_all=None):
    # parse a condition, either filter predicate or join operation
    start = None
    join = False
    join_keys = {}
    cond = cond.replace(" in ", " IN ")
    cond = cond.replace(" not in ", " NOT IN ")
    cond = cond.replace(" like ", " LIKE ")
    cond = cond.replace(" not like ", " NOT LIKE ")
    cond = cond.replace(" between ", " BETWEEN ")
    s = None
    ops = None
    
    if ' IN ' in cond:
        s = cond.split(' IN ')
        ops = "in"
    elif " NOT IN " in cond:
        s = cond.split(' NOT IN ')
        ops = "not in"
    elif " LIKE " in cond:
        s = cond.split(' LIKE ')
        ops = "like"
    elif " NOT LIKE " in cond:
        s = cond.split(' NOT LIKE ')
        ops = "not like"
    elif " BETWEEN " in cond:
        s = cond.split(' BETWEEN ')
        ops = "between"
    elif " IS " in cond:
        s = cond.split(' IS ')
        ops = OPS["="]
        
        
    if ' IN ' in cond or " NOT IN " in cond:
        attr = s[0].strip()
        try:
            value = list(ast.literal_eval(s[1].strip()))
        except:
            temp_value = s[1].strip()[1:][:-1].split(',')
            value = []
            for v in temp_value:
                value.append(v.strip())
        if tables_all:
            table = tables_all[attr.split(".")[0].strip()]
            attr = table + "." + attr.split(".")[-1].strip()
        else:
            table = attr.split(".")[0].strip()
        return table, [attr, ops, value], join, join_keys
    
    elif s is not None:
        attr = s[0].strip()
        value = s[1].strip()
        if tables_all:
            table = tables_all[attr.split(".")[0].strip()]
            attr = table + "." + attr.split(".")[-1].strip()
        else:
            table = attr.split(".")[0].strip()
        return table, [attr, ops, value], join, join_keys
    
        

    for i in range(len(cond)):
        s = cond[i]
        if s in OPS:
            start = i
            if cond[i + 1] in OPS:
                end = i + 2
            else:
                end = i + 1
            break
    
    if start is None:
        return None, [None, None, None], join, join_keys
    assert start is not None
    left = cond[:start].strip()
    ops = cond[start:end].strip()
    right = cond[end:].strip()
    table1 = left.split(".")[0].strip().lower()
    if tables_all:
        cond = cond.replace(table1 + ".", tables_all[table1] + ".")
        table1 = tables_all[table1]
        left = table1 + "." + left.split(".")[-1].strip()
    if "." in right:
        table2 = right.split(".")[0].strip().lower()
        if table2 in tables_all:
            cond = cond.replace(table2 + ".", tables_all[table2] + ".")
            table2 = tables_all[table2]
            right = table2 + "." + right.split(".")[-1].strip()
            join = True
            join_keys[table1] = left
            join_keys[table2] = right
            return table1 + " " + table2, cond, join, join_keys

    value = right.strip()
    if value[0] == "'" and value[-1] == "'":
        value = value[1:-1]
    try:
        value = list(ast.literal_eval(value.strip()))
    except:
        try:
            value = int(value)
        except:
            try:
                value = float(value)
            except:
                value = value

    return table1, [left, ops, value], join, join_keys

In [11]:
def process_condition_join(cond, tables_all):
    start = None
    join = False
    join_keys = {}
    for i in range(len(cond)):
        s = cond[i]
        if s == "=":
            start = i
            if cond[i + 1] == "=":
                end = i + 2
            else:
                end = i + 1
            break
    
    if start is None:
        return None, None, False, None
    
    left = cond[:start].strip()
    ops = cond[start:end].strip()
    right = cond[end:].strip()
    table1 = left.split(".")[0].strip().lower()
    if table1 in tables_all:
        left = tables_all[table1] + "." + left.split(".")[-1].strip()
    else:
        assert False, table1
        return None, None, False, None
    if "." in right:
        table2 = right.split(".")[0].strip().lower()
        if table2 in tables_all:
            right = tables_all[table2] + "." + right.split(".")[-1].strip()
            join = True
            join_keys[table1] = left
            join_keys[table2] = right
            return table1 + " " + table2, left + " = " + right, join, join_keys
    return None, None, False, None
    
    

def parse_query_all_join(query):
    """
    This function will parse out all join conditions from the query. 
    """
    query = query.replace(" where ", " WHERE ")
    query = query.replace(" from ", " FROM ")
    #query = query.replace(" and ", " AND ")
    query = query.split(";")[0]
    query = query.strip()
    tables_all = {}
    join_cond = []
    join_keys = {}
    tables_str = query.split(" WHERE ")[0].split(" FROM ")[-1]
    for table_str in tables_str.split(","):
        table_str = table_str.strip()
        if " as " in table_str:
            tables_all[table_str.split(" as ")[-1]] = table_str.split(" as ")[0]
        else:
            tables_all[table_str.split(" ")[-1]] = table_str.split(" ")[0]
    # processing conditions
    conditions = query.split(" WHERE ")[-1].split(" AND ")
    for cond in conditions:
        cond = cond.strip()
        if cond[0] == "(" and cond[-1] == ")":
            cond = cond[1:-1]
        table, cond, join, join_key = process_condition_join(cond, tables_all)
        
        if join:
            join_cond.append(cond)
            for tab in join_key:
                if tab in join_keys:
                    join_keys[tab].add(join_key[tab])
                else:
                    join_keys[tab] = set([join_key[tab]])

    return tables_all, join_cond, join_keys

def get_join_hyper_graph(join_keys, equivalent_keys):
    equivalent_group = dict()
    for table in join_keys:
        for key in join_keys[table]:
            seen = False
            for indicator in equivalent_keys:
                if key in equivalent_keys[indicator]:
                    if seen:
                        assert False, f"{key} appears in multiple equivalent groups."
                    if indicator not in equivalent_group:
                        equivalent_group[indicator] = [key]
                    else:
                        equivalent_group[indicator].append(key)
                    seen = True
            if not seen:
                assert False, f"no equivalent groups found for {key}."
    return equivalent_group

In [4]:
data_path = "/home/ubuntu/data_CE/{}.csv"
schema = gen_imdb_schema(data_path)
BN = Bound_ensemble(None, None, schema)

In [5]:
query_path = "/home/ubuntu/data_CE/job/"
queries = dict()
for file in os.listdir(query_path):
    if file.endswith(".sql"):
        with open(query_path+file, "r") as f:
            q = f.readline()
            queries[file.split(".sql")[0]] = q

In [6]:
import numpy as np
import copy


class Bucket:
    """
    The class of bucketization of a key attribute
    """

    def __init__(self, name, bins=[], bin_modes=[], bin_vars=[], bin_means=[], rest_bins_remaining=None):
        self.name = name
        self.bins = bins
        self.bin_modes = bin_modes
        self.bin_vars = bin_vars
        self.bin_means = bin_means
        self.rest_bins_remaining = rest_bins_remaining
        if len(bins) != 0:
            assert len(bins) == len(bin_modes)


class Table_bucket:
    """
    The class of bucketization for all key attributes in a table.
    Supporting more than three dimensional bin modes requires simplifying the causal structure, which is left as a
    future work.
    """
    def __init__(self, table_name, id_attributes, bin_sizes):
        self.table_name = table_name
        self.id_attributes = id_attributes
        self.bin_sizes = bin_sizes
        self.oned_bin_modes = dict()
        self.twod_bin_modes = dict()


class Bucket_group:
    """
    The class of bucketization for a group of equivalent join keys
    """

    def __init__(self, buckets, start_key, sample_rate, bins=None, primary_keys=[]):
        self.buckets = buckets
        self.start_key = start_key
        self.sample_rate = sample_rate
        self.bins = bins
        self.primary_keys = primary_keys

    def bucketize(self, data):
        """
        Discretize data based on the bucket
        """
        res = dict()
        seen_remain_key = np.array([])
        cumulative_bin = copy.deepcopy(self.buckets[self.start_key].bins)
        start_means = np.asarray(self.buckets[self.start_key].bin_means)

        for key in data:
            if key in self.primary_keys:
                continue
            res[key] = copy.deepcopy(data[key])
            if key != self.start_key:
                unique_remain = np.setdiff1d(self.buckets[key].rest_bins_remaining, seen_remain_key)
                assert sum([np.sum(np.isin(unique_remain, b) == 1) for b in cumulative_bin]) == 0

                if len(unique_remain) != 0:
                    remaining_data = data[key][np.isin(data[key], unique_remain)]
                    unique_remain, count_remain = np.unique(remaining_data, return_counts=True)
                    unique_counts = np.unique(count_remain)
                    for u in unique_counts:
                        temp_idx = np.searchsorted(start_means, u)
                        if temp_idx == len(cumulative_bin):
                            idx = -1
                            if u > self.buckets[key].bin_modes[-1]:
                                self.buckets[key].bin_modes[-1] = u
                        elif temp_idx == 0:
                            idx = 0
                        else:
                            if (u - start_means[temp_idx - 1]) >= (start_means[temp_idx] - u):
                                idx = temp_idx - 1
                            else:
                                idx = temp_idx
                        temp_unique = unique_remain[count_remain == u]
                        cumulative_bin[idx] = np.concatenate((cumulative_bin[idx], temp_unique))
                        seen_remain_key = np.concatenate((seen_remain_key, temp_unique))
                        if u > self.buckets[key].bin_modes[idx]:
                            self.buckets[key].bin_modes[idx] = u
            res[key] = copy.deepcopy(data[key])
            count = 0
            for i, b in enumerate(cumulative_bin):
                count += len(data[key][np.isin(data[key], b)])
                res[key][np.isin(data[key], b)] = i

            if self.sample_rate[key] < 1.0:
                bin_modes = self.buckets[key].bin_modes
                bin_modes[bin_modes != 1] = bin_modes[bin_modes != 1] / self.sample_rate[key]
                self.buckets[key].bin_modes = bin_modes

        self.bins = cumulative_bin

        for key in data:
            if key in self.primary_keys:
                res[key] = self.bucketize_PK(data[key])
                self.buckets[key] = Bucket(key)
        return res

    def bucketize_PK(self, data):
        res = copy.deepcopy(data)
        remaining_data = np.unique(data)
        for i, b in enumerate(self.bins):
            res[np.isin(data, b)] = i
            remaining_data = np.setdiff1d(remaining_data, b)
        if len(remaining_data) != 0:
            self.bins.append(list(remaining_data))
            for key in self.buckets:
                if key not in self.primary_keys:
                    self.buckets[key].bin_modes = np.append(self.buckets[key].bin_modes, 0)
        res[np.isin(data, remaining_data)] = len(self.bins)
        return res



def identify_key_values(schema):
    """
    identify all the key attributes from the schema of a DB, currently we assume all possible joins are known
    It is also easy to support unseen joins, which we left as a future work.
    :param schema: the schema of a DB
    :return: a dict of all keys, {table: [keys]};
             a dict of set, each indicating which keys on different tables are considered the same key.
    """
    all_keys = set()
    equivalent_keys = dict()
    for i, join in enumerate(schema.relationships):
        keys = join.identifier.split(" = ")
        all_keys.add(keys[0])
        all_keys.add(keys[1])
        seen = False
        for k in equivalent_keys:
            if keys[0] in equivalent_keys[k]:
                equivalent_keys[k].add(keys[1])
                seen = True
                break
            elif keys[1] in equivalent_keys[k]:
                equivalent_keys[k].add(keys[0])
                seen = True
                break
        if not seen:
            # set the keys[-1] as the identifier of this equivalent join key group for convenience.
            equivalent_keys[keys[-1]] = set(keys)

    assert len(all_keys) == sum([len(equivalent_keys[k]) for k in equivalent_keys])
    return all_keys, equivalent_keys


def equal_freq_binning(name, data, n_bins, data_len):
    uniques, counts = data
    if len(uniques) <= n_bins:
        bins = []
        bin_modes = []
        bin_vars = []
        bin_means = []
        
        for i, uni in enumerate(uniques):
            bins.append([uni])
            bin_modes.append(counts[i])
            bin_vars.append(0)
            bin_means.append(counts[i])
        return Bucket(name, bins, bin_modes, bin_vars, bin_means)
    
    unique_counts, count_counts = np.unique(counts, return_counts=True)
    idx = np.argsort(unique_counts)
    unique_counts = unique_counts[idx]
    count_counts = count_counts[idx]

    bins = []
    bin_modes = []
    bin_vars = []
    bin_means = []

    bin_freq = data_len / n_bins
    cur_freq = 0
    cur_bin = []
    cur_bin_count = []
    for i, uni_c in enumerate(unique_counts):
        cur_freq += count_counts[i] * uni_c
        cur_bin.append(uniques[np.where(counts == uni_c)[0]])
        cur_bin_count.extend([uni_c] * count_counts[i])
        if (cur_freq > bin_freq) or (i == (len(unique_counts) - 1)):
            bins.append(np.concatenate(cur_bin))
            cur_bin_count = np.asarray(cur_bin_count)
            bin_modes.append(uni_c)
            bin_means.append(np.mean(cur_bin_count))
            bin_vars.append(np.var(cur_bin_count))
            cur_freq = 0
            cur_bin = []
            cur_bin_count = []
    assert len(uniques) == sum([len(b) for b in bins]), f"some unique values missed or duplicated"
    return Bucket(name, bins, bin_modes, bin_vars, bin_means)


def compute_variance_score(buckets):
    """
    compute the variance of products of random variables
    """
    all_mean = np.asarray([buckets[k].bin_means for k in buckets])
    all_var = np.asarray([buckets[k].bin_vars for k in buckets])
    return np.sum(np.prod(all_var + all_mean ** 2, axis=0) - np.prod(all_mean, axis=0) ** 2)


def sub_optimal_bucketize(data, sample_rate, n_bins=30, primary_keys=[]):
    """
    Perform sub-optimal bucketization on a group of equivalent join keys.
    :param data: a dict of (potentially sampled) table data of the keys
                 the keys of this dict are one group of equivalent join keys
    :param sample_rate: the sampling rate the data, could be all 1 if no sampling is performed
    :param n_bins: how many bins can we allocate
    :param primary_keys: the primary keys in the equivalent group since we don't need to bucketize PK.
    :return: new data, where the keys are bucketized
             the mode of each bucket
    """
    unique_values = dict()
    for key in data:
        if key not in primary_keys:
            unique_values[key] = np.unique(data[key], return_counts=True)

    best_variance_score = np.infty
    best_bin_len = 0
    best_start_key = None
    best_buckets = None
    for start_key in data:
        if start_key in primary_keys:
            continue
        start_bucket = equal_freq_binning(start_key, unique_values[start_key], n_bins, len(data[start_key]))
        rest_buckets = dict()
        for key in data:
            if key == start_key or key in primary_keys:
                continue
            uniques = unique_values[key][0]
            counts = unique_values[key][1]
            rest_buckets[key] = Bucket(key, [], [0] * len(start_bucket.bins), [0] * len(start_bucket.bins),
                                       [0] * len(start_bucket.bins), uniques)
            for i, bin in enumerate(start_bucket.bins):
                idx = np.where(np.isin(uniques, bin) == 1)[0]
                if len(idx) != 0:
                    bin_count = counts[idx]
                    unique_bin_keys = uniques[idx]
                    # unique_bin_count = np.unique(bin_count)
                    # bin_count = np.concatenate([counts[counts == j] for j in unique_bin_count])
                    # unique_bin_keys = np.concatenate([uniques[counts == j] for j in unique_bin_count])
                    rest_buckets[key].rest_bins_remaining = np.setdiff1d(rest_buckets[key].rest_bins_remaining,
                                                                         unique_bin_keys)
                    rest_buckets[key].bin_modes[i] = np.max(bin_count)
                    rest_buckets[key].bin_vars[i] = np.var(bin_count)
                    rest_buckets[key].bin_means[i] = np.mean(bin_count)

        rest_buckets[start_key] = start_bucket
        var_score = compute_variance_score(rest_buckets)
        if len(start_bucket.bins) > best_bin_len:
            best_variance_score = var_score
            best_start_key = start_key
            best_buckets = rest_buckets
            best_bin_len = len(start_bucket.bins)
        elif len(start_bucket.bins) >= best_bin_len * 0.9 and var_score < best_variance_score:
            best_variance_score = var_score
            best_start_key = start_key
            best_buckets = rest_buckets
            best_bin_len = len(start_bucket.bins)

    best_buckets = Bucket_group(best_buckets, best_start_key, sample_rate, primary_keys=primary_keys)
    new_data = best_buckets.bucketize(data)
    return new_data, best_buckets


def fixed_start_key_bucketize(start_key, data, sample_rate, n_bins=30, primary_keys=[]):
    """
    Perform sub-optimal bucketization on a group of equivalent join keys based on the pre-defined start_key.
    :param data: a dict of (potentially sampled) table data of the keys
                 the keys of this dict are one group of equivalent join keys
    :param sample_rate: the sampling rate the data, could be all 1 if no sampling is performed
    :param n_bins: how many bins can we allocate
    :param primary_keys: the primary keys in the equivalent group since we don't need to bucketize PK.
    :return: new data, where the keys are bucketized
             the mode of each bucket
    """
    unique_values = dict()
    for key in data:
        if key not in primary_keys:
            unique_values[key] = np.unique(data[key], return_counts=True)

    start_bucket = equal_freq_binning(start_key, unique_values[start_key], n_bins, len(data[start_key]))
    rest_buckets = dict()
    for key in data:
        if key == start_key or key in primary_keys:
            continue
        uniques = unique_values[key][0]
        counts = unique_values[key][1]
        rest_buckets[key] = Bucket(key, [], [0] * len(start_bucket.bins), [0] * len(start_bucket.bins),
                                   [0] * len(start_bucket.bins), uniques)
        for i, bin in enumerate(start_bucket.bins):
            idx = np.where(np.isin(uniques, bin) == 1)[0]
            if len(idx) != 0:
                bin_count = counts[idx]
                unique_bin_keys = uniques[idx]
                rest_buckets[key].rest_bins_remaining = np.setdiff1d(rest_buckets[key].rest_bins_remaining,
                                                                     unique_bin_keys)
                rest_buckets[key].bin_modes[i] = np.max(bin_count)
                rest_buckets[key].bin_means[i] = np.mean(bin_count)

    best_buckets = Bucket_group(rest_buckets, start_key, sample_rate, primary_keys=primary_keys)
    new_data = best_buckets.bucketize(data, n_bins)
    return new_data, best_buckets



In [7]:
all_keys, equivalent_keys = identify_key_values(schema)
print(equivalent_keys)
print(len(equivalent_keys))

{'kind_type.id': {'aka_title.kind_id', 'kind_type.id', 'title.kind_id'}, 'info_type.id': {'movie_info.info_type_id', 'info_type.id', 'person_info.info_type_id', 'movie_info_idx.info_type_id'}, 'title.id': {'cast_info.movie_id', 'movie_info_idx.movie_id', 'complete_cast.movie_id', 'movie_info.movie_id', 'movie_keyword.movie_id', 'aka_title.movie_id', 'title.id', 'movie_companies.movie_id'}, 'name.id': {'name.id', 'person_info.person_id', 'cast_info.person_id', 'aka_name.person_id'}, 'char_name.id': {'cast_info.person_role_id', 'char_name.id'}, 'role_type.id': {'cast_info.role_id', 'role_type.id'}, 'comp_cast_type.id': {'complete_cast.subject_id', 'complete_cast.status_id', 'comp_cast_type.id'}, 'keyword.id': {'keyword.id', 'movie_keyword.keyword_id'}, 'company_name.id': {'company_name.id', 'movie_companies.company_id'}, 'company_type.id': {'movie_companies.company_type_id', 'company_type.id'}}
10


In [8]:
all_join_keys_stats = dict()
for key in queries:
    q = queries[key]
    res = parse_query_all_join(q)
    for table in res[-1]:
        for join_key in list(res[-1][table]):
            for PK in equivalent_keys:
                if join_key in equivalent_keys[PK]:
                    if PK in all_join_keys_stats:
                        all_join_keys_stats[PK] += 1
                    else:
                        all_join_keys_stats[PK] = 1
                    break
                

In [12]:
tables_all, join_cond, join_keys = parse_query_all_join(queries['29b'])
get_join_hyper_graph(join_keys, equivalent_keys)

{'title.id': ['title.id',
  'movie_info.movie_id',
  'movie_companies.movie_id',
  'cast_info.movie_id',
  'movie_keyword.movie_id',
  'complete_cast.movie_id'],
 'info_type.id': ['movie_info.info_type_id',
  'info_type.id',
  'person_info.info_type_id'],
 'company_name.id': ['movie_companies.company_id', 'company_name.id'],
 'role_type.id': ['cast_info.role_id', 'role_type.id'],
 'name.id': ['cast_info.person_id',
  'name.id',
  'aka_name.person_id',
  'person_info.person_id'],
 'char_name.id': ['cast_info.person_role_id', 'char_name.id'],
 'keyword.id': ['movie_keyword.keyword_id', 'keyword.id'],
 'comp_cast_type.id': ['complete_cast.subject_id',
  'complete_cast.status_id',
  'comp_cast_type.id']}

In [13]:
all_join_keys_stats

{'title.id': 447,
 'info_type.id': 160,
 'company_name.id': 142,
 'char_name.id': 44,
 'name.id': 138,
 'role_type.id': 40,
 'char_name.id': 44,
 'keyword.id': 150,
 'comp_cast_type.id': 60,
 'kind_type.id': 52,
 'company_type.id': 82}

In [14]:
n_bins = {
    'title.id': 800,
    'info_type.id': 100,
    'keyword.id': 100,
    'company_name.id': 100,
    'name.id': 100,
    'company_type.id': 100,
    'comp_cast_type.id': 50,
    'kind_type.id': 50,
    'char_name.id': 50,
    'role_type.id': 50
}

In [15]:
def make_sample(np_data, nrows=1000000, seed=0):
    np.random.seed(seed)
    if len(np_data) <= nrows:
        return np_data, 1.0
    else:
        selected = np.random.choice(len(np_data), size=nrows, replace=False)
        return np_data[selected], nrows/len(np_data)

def stats_analysis(sample, data, sample_rate, show=10):
    n, c = np.unique(sample, return_counts=True)
    idx = np.argsort(c)[::-1]
    for i in range(min(show, len(idx))):
        print(c[idx[i]], c[idx[i]]/sample_rate, len(data[data == n[idx[i]]]))

In [16]:
data = dict()
sample_rate = dict()
primary_keys = []
for table_obj in schema.tables:
    df_rows = pd.read_csv(table_obj.csv_file_location, header=None, escapechar='\\', encoding='utf-8', quotechar='"',
                          sep=",")
    
    df_rows.columns = [table_obj.table_name + '.' + attr for attr in table_obj.attributes]

    for attribute in table_obj.irrelevant_attributes:
        df_rows = df_rows.drop(table_obj.table_name + '.' + attribute, axis=1)

    df_rows.apply(pd.to_numeric, errors="ignore")
    for attr in df_rows.columns:
        if attr in all_keys:
            print(attr)
            print(np.sum(np.isnan(df_rows[attr].values)))
            data[attr] = df_rows[attr].values
            data[attr][np.isnan(data[attr])] = -1
            data[attr][data[attr] < 0] = -1
            print(len(data[attr]), len(df_rows[attr].values))
            data[attr] = copy.deepcopy(data[attr])[data[attr]>=0]
            print(len(data[attr]), len(df_rows[attr].values))
            if len(np.unique(data[attr])) >= len(data[attr]) - 10:
                primary_keys.append(attr)
            print(np.sum(np.isnan(df_rows[attr].values)))

/home/ubuntu/miniconda3/envs/bayescard/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (5,10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


title.id
0
2528312 2528312
2528312 2528312
0
title.kind_id
0
2528312 2528312
2528312 2528312
0
movie_info_idx.movie_id
0
1380035 1380035
1380035 1380035
0
movie_info_idx.info_type_id
0
1380035 1380035
1380035 1380035
0


/home/ubuntu/miniconda3/envs/bayescard/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


movie_info.movie_id
0
14835720 14835720
14835720 14835720
0
movie_info.info_type_id
0
14835720 14835720
14835720 14835720
0
info_type.id
0
113 113
113 113
0
cast_info.person_id
0
36244344 36244344
36244344 36244344
0
cast_info.movie_id
0
36244344 36244344
36244344 36244344
0
cast_info.person_role_id
18672825
36244344 36244344
17571519 36244344
0
cast_info.role_id
0
36244344 36244344
36244344 36244344
0


/home/ubuntu/miniconda3/envs/bayescard/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


char_name.id
0
3140339 3140339
3140339 3140339
0
role_type.id
0
12 12
12 12
0
complete_cast.movie_id
0
135086 135086
135086 135086
0
complete_cast.subject_id
0
135086 135086
135086 135086
0
complete_cast.status_id
0
135086 135086
135086 135086
0
comp_cast_type.id
0
4 4
4 4
0
name.id
0
4167491 4167491
4167491 4167491
0


/home/ubuntu/miniconda3/envs/bayescard/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3173: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


aka_name.person_id
0
901343 901343
901343 901343
0
movie_keyword.movie_id
0
4523930 4523930
4523930 4523930
0
movie_keyword.keyword_id
0
4523930 4523930
4523930 4523930
0
keyword.id
0
134170 134170
134170 134170
0
person_info.person_id
0
2963664 2963664
2963664 2963664
0
person_info.info_type_id
0
2963664 2963664
2963664 2963664
0
movie_companies.movie_id
0
2609129 2609129
2609129 2609129
0
movie_companies.company_id
0
2609129 2609129
2609129 2609129
0
movie_companies.company_type_id
0
2609129 2609129
2609129 2609129
0
company_name.id
0
234997 234997
234997 234997
0
company_type.id
0
4 4
4 4
0
aka_title.movie_id
0
361472 361472
361472 361472
0
aka_title.kind_id
0
361472 361472
361472 361472
0
kind_type.id
0
7 7
7 7
0


In [17]:
sample_rate = dict()
sampled_data = dict()
for k in data:
    print(k)
    temp = make_sample(data[k], 1000000)
    stats_analysis(temp[0], data[k], temp[1])
    sampled_data[k] = temp[0]
    sample_rate[k] = temp[1]


title.id
1 2.528312 1
1 2.528312 1
1 2.528312 1
1 2.528312 1
1 2.528312 1
1 2.528312 1
1 2.528312 1
1 2.528312 1
1 2.528312 1
1 2.528312 1
title.kind_id
609674 1541446.090288 1543264
262591 663911.976392 662824
46890 118552.54968 118234
39888 100849.309056 100537
35940 90867.53328 90852
5016 12682.012992 12600
1 2.528312 1
movie_info_idx.movie_id
4 5.52014 4
4 5.52014 4
4 5.52014 4
4 5.52014 4
4 5.52014 4
4 5.52014 4
4 5.52014 4
4 5.52014 4
4 5.52014 4
4 5.52014 4
movie_info_idx.info_type_id
333438 460156.11033 459925
333295 459958.765325 459925
333076 459656.53766 459925
185 255.30647499999998 250
6 8.28021 10
movie_info.movie_id
194 2878.12968 2937
118 1750.61496 1584
109 1617.09348 1638
97 1439.06484 1344
92 1364.88624 1281
89 1320.37908 1081
83 1231.36476 1219
82 1216.5290400000001 1160
77 1142.35044 1189
77 1142.35044 1207
movie_info.info_type_id
204500 3033904.74 3036719
103748 1539176.27856 1533909
94821 1406737.80612 1401902
89252 1324117.68144 1325361
87478 1297799.11416 12989

In [ ]:
len(sampled_data)

In [ ]:
np.unique(sampled_data["kind_type.id"])

In [ ]:
print(len(equivalent_keys))
print(len(temp))

In [ ]:
equivalent_keys

In [18]:
temp = dict()
for PK in equivalent_keys:
    #if PK != 'kind_type.id':
     #   continue
    print(equivalent_keys[PK])
    group_data = {}
    group_sample_rate = {}
    for K in equivalent_keys[PK]:
        group_data[K] = sampled_data[K]
        group_sample_rate[K] = sample_rate[K]
    d, bucket = sub_optimal_bucketize(group_data, sample_rate, n_bins=n_bins[PK], primary_keys=primary_keys)
    temp[PK] = bucket

{'kind_type.id', 'aka_title.kind_id', 'title.kind_id'}
{'info_type.id', 'person_info.info_type_id', 'movie_info_idx.info_type_id', 'movie_info.info_type_id'}
{'movie_keyword.movie_id', 'movie_companies.movie_id', 'complete_cast.movie_id', 'movie_info_idx.movie_id', 'movie_info.movie_id', 'cast_info.movie_id', 'title.id', 'aka_title.movie_id'}
{'aka_name.person_id', 'cast_info.person_id', 'name.id', 'person_info.person_id'}
{'cast_info.person_role_id', 'char_name.id'}
{'role_type.id', 'cast_info.role_id'}
{'comp_cast_type.id', 'complete_cast.subject_id', 'complete_cast.status_id'}
{'keyword.id', 'movie_keyword.keyword_id'}
{'movie_companies.company_id', 'company_name.id'}
{'company_type.id', 'movie_companies.company_type_id'}


In [ ]:
sampled_data['kind_type.id']

In [ ]:
bucket.bins

In [19]:
temp_bins = dict()
for key in temp:
    temp_bins[key] = temp[key].bins

In [21]:
for k in temp_bins:
    print(len(temp_bins[k]))

8
72
74
29
28
12
2
86
75
3


In [20]:
import pickle
with open("bins.pkl", "wb") as f:
    pickle.dump(temp_bins, f, pickle.HIGHEST_PROTOCOL)
with open("equivalent_keys.pkl", "wb") as f:
    pickle.dump(equivalent_keys, f, pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open("bins.pkl", "rb") as f:
    bins = pickle.load(f)
with open("equivalent_keys.pkl", "rb") as f:
    equivalent_keys = pickle.load(f)

In [ ]:
equivalent_keys

In [ ]:
[len(temp_bins[k]) for k in temp_bins]

In [ ]:
bucket = temp['company_type.id']
for k in bucket.buckets:
    print("==============================================================")
    print(k, len(bucket.buckets[k].bin_modes), len(bucket.bins))
    print(bucket.buckets[k].bin_modes)
    print([len(b) for b in bucket.buckets[k].bins])
    print(bucket.buckets[k].bin_means)
    print(bucket.buckets[k].bin_vars)

In [ ]:
data_path = "/Users/ziniuw/Desktop/past_research/End-to-End-CardEst-Benchmark/datasets/stats_simplified/{}.csv"
model_folder = "../../CE_scheme_models"
data, null_values, key_attrs, table_buckets, equivalent_keys, schema, bin_size = process_stats_data(data_path,
                                                                       model_folder, 200, False)

In [ ]:
table = "votes"
bucket = table_buckets[table]
for attr in bucket.id_attributes:
    print(attr)
    print(np.sum(bucket.oned_bin_modes[attr]), bucket.oned_bin_modes[attr].shape)
    if len(bucket.twod_bin_modes[attr]) != 0:
        print(np.sum(bucket.twod_bin_modes[attr]), bucket.twod_bin_modes[attr].shape)

In [ ]:
table = "postLinks"
bucket = table_buckets[table]
for attr in bucket.id_attributes:
    print(attr)
    print(np.sum(bucket.oned_bin_modes[attr]), bucket.oned_bin_modes[attr].shape)
    if len(bucket.twod_bin_modes[attr]) != 0:
        print(np.sum(bucket.twod_bin_modes[attr]), bucket.twod_bin_modes[attr].shape)

In [ ]:
print(bucket.oned_bin_modes[bucket.id_attributes[0]])
print(np.sum(bucket.twod_bin_modes[bucket.id_attributes[0]], axis=1))
print(bucket.oned_bin_modes[bucket.id_attributes[1]])
print(np.sum(bucket.twod_bin_modes[bucket.id_attributes[1]], axis=0))